In [10]:
pip install pdfplumber pyspark openai

  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached anyio-4.6.2.post1-py3-none-any.whl (90 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.0 MB/s eta 0:00:00
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [11]:
import pdfplumber
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Initialize Spark session
spark = SparkSession.builder.appName("PDFToSpark").getOrCreate()

# Define schema for the Spark DataFrame
schema = StructType([
    StructField("Card Name", StringType(), True),
    StructField("Bank", StringType(), True),
    StructField("Annual Fee", StringType(), True),  # Keep as String to accommodate "No annual fee"
    StructField("Purchase Interest Rate (%)", FloatType(), True),
    StructField("Rewards", StringType(), True)
])

def extract_pdf_data(file_path):
    """
    Extracts data from the credit card PDF and structures it for Spark DataFrame.
    """
    data = []
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            lines = text.split("\n")
            
            # Parse lines for credit card information
            for i in range(len(lines)):
                if "%" in lines[i]:  # Heuristic: look for interest rate
                    # Extract card details (sample parsing logic, adjust as needed)
                    try:
                        card_name = lines[i - 3].strip()
                        bank = lines[i - 2].strip()
                        annual_fee = lines[i + 1].split(" ")[0].strip() if "fee" in lines[i + 1].lower() else "No annual fee"
                        interest_rate = float(lines[i].split("%")[0].strip())
                        rewards = None
                        
                        # Check for rewards (look at subsequent lines)
                        rewards_lines = []
                        j = i + 2
                        while j < len(lines) and ":" in lines[j]:
                            rewards_lines.append(lines[j].split(":")[1].strip())
                            j += 1
                        
                        if rewards_lines:
                            rewards = ", ".join(rewards_lines)
                        
                        # Append to data
                        data.append((card_name, bank, annual_fee, interest_rate, rewards))
                    except Exception as e:
                        print(f"Skipping line due to parsing error: {e}")
                        continue
    return data

# Path to the PDF file
pdf_path = "/Users/aaryas127/Documents/GitHub/credit_card_reward_maximizer/SearchCreditCard-eng.pdf"

# Extract data from the PDF
parsed_data = extract_pdf_data(pdf_path)

# Create a Spark DataFrame from the parsed data
df = spark.createDataFrame(parsed_data, schema=schema)

# Show the DataFrame
df.show(truncate=False)


+------------------------------------------------------------+-----------------------------------------------+-------------+--------------------------+-------+
|Card Name                                                   |Bank                                           |Annual Fee   |Purchase Interest Rate (%)|Rewards|
+------------------------------------------------------------+-----------------------------------------------+-------------+--------------------------+-------+
|Desjardins Flexi Visa                                       |Desjardins                                     |No annual fee|10.9                      |NULL   |
|MBNA True Line® Mastercard®                                 |MBNA is a division of The Toronto-Dominion Bank|No annual fee|12.99                     |NULL   |
|Coast Capital Collabria Classic Mastercard                  |Coast Capital Savings                          |No annual fee|13.99                     |NULL   |
|Tangerine Money-Back Credit Card       

In [12]:
df.first()["Card Name"]

'Desjardins Flexi Visa'

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

In [ ]:
# Cashback

from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "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"
)

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Does the Desjardins Flexi Visa have cashbacks?"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

According to the Desjardins website, the Desjardins Flexi Visa card does offer cashback rewards, but the specifics may vary depending on the card's terms and conditions.

Here's what I found:

* The Desjardins Flexi Visa card offers 1% cashback on most purchases, with no rotating categories or spending limits.
* You can also earn bonus cashback rates on certain categories, such as:
	+ 2% cashback on grocery purchases (some conditions apply)
	+ 2% cashback on recurring bill payments (some conditions apply)
* Cashback rewards are credited to your account annually, in January of each year.

Please note that these details are subject to change, and you should always check the Desjardins website or consult with a representative for the most up-to-date information on the Flexi Visa card's cashback rewards program.

Would you like me to look into any other features or benefits of the Desjardins Flexi Visa card?

In [19]:
# Travel Benefits

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Does the Desjardins Flexi Visa have any travel benefits?"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

The Desjardins Flexi Visa card does offer some travel-related benefits. Here are a few:

1. **Travel Insurance**: The card comes with travel insurance that covers you for up to 3 days, including trip cancellation, interruption, and delay insurance, as well as travel medical insurance.
2. **Travel Assistance**: You'll have access to a 24/7 travel assistance service, which can help you with things like flight reservations, hotel bookings, and travel documentation.
3. **Purchase Protection and Extended Warranty**: While not exclusively a travel benefit, the card's purchase protection and extended warranty can provide peace of mind when making travel-related purchases, such as luggage or travel gear.
4. **Travel Discounts**: As a Desjardins member, you may be eligible for discounts on travel packages, flights, and hotels through Desjardins' travel agency, Desjardins Travel.

It's worth noting that the Desjardins Flexi Visa card is a basic, no-fee credit card, so the travel benefits are not

In [20]:
# Signup Bonus, Points, Spending requirement

completion = client.chat.completions.create(
  model="meta/llama-3.1-405b-instruct",
  messages=[{"role":"user","content":"Does the Desjardins Flexi Visa have the ability to get you any sort of points?"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

According to the Desjardins website, the Desjardins Flexi Visa card does offer a rewards program, but it's a bit different from traditional points-based programs.

With the Desjardins Flexi Visa, you can earn BONUSDOLLARS, which are redeemable for cash back, travel, or other rewards. Here's how it works:

* You earn 1% BONUSDOLLARS on all purchases made with your card.
* You can redeem your BONUSDOLLARS for:
	+ Cash back: deposited directly into your Desjardins account
	+ Travel: booked through the Desjardins Travel Agency or online
	+ Other rewards: such as gift cards, merchandise, or charitable donations

While the Desjardins Flexi Visa doesn't offer traditional points like some other credit cards, the BONUSDOLLARS program still provides a way to earn rewards on your purchases.

Keep in mind that the rewards program may be subject to change, and there may be some conditions or restrictions on redeeming your BONUSDOLLARS. It's always a good idea to review the terms and conditions of y

In [ ]:
# Insurance

In [ ]:
# Annual fee...first year free?